In [65]:
import pandas as pd

In [80]:
gni_capita = pd.read_csv("gnipercapita_ppp_current_international.csv")
life_exp = pd.read_csv("life_expectancy_years.csv")
population = pd.read_csv("population_total.csv")

There are three things, you have to do, before building the dashboard.

- The data contains missing values. For all three CSVs, use forward filling for imputing the missing data. Forward filling uses earlier values of the country for filling later ones (tip: there exist a Pandas function for that)
- For each CSV, transform the dataframe into a so called "tidy data format" (see e.g. https://en.wikipedia.org/wiki/Tidy_data). Each dataframe then should have only three columns: (1) country, (2) year, (3) the KPI (that means either a) life expectancy, b) population, or 3) GNI per capita)
- You have to merge all three CVS. Merge the three dataframe into one dataframe, that just has 5 columns (country, year and all three KPIs from the three dataframes)


In [81]:
gni_capita = gni_capita.set_index("country", drop=True)
gni_capita[["1990"]] = gni_capita[["1990"]].fillna(0, axis=0)
gni_capita = gni_capita.ffill(axis=1)
gni_capita = (gni_capita.replace({'k':'*1000'}, regex=True).dropna().apply(pd.eval, axis=1, result_type="expand"))
gni_capita.columns = [i for i in range(1990,2021)]
gni_capita = pd.DataFrame(gni_capita.stack())
gni_capita.index = gni_capita.index.rename('year', level=1)
gni_capita.columns = ["GNI per capita"]
gni_capita = gni_capita.reset_index()

In [83]:
life_exp = life_exp.set_index("country", drop=True)
life_exp[["1800"]] = life_exp[["1800"]].fillna(0, axis=0)
life_exp = life_exp.ffill(axis=1)
life_exp.columns = [i for i in range(1800,2101)]
life_exp = pd.DataFrame(life_exp.stack())
life_exp.index = life_exp.index.rename('year', level=1)
life_exp.columns = ["life expectancy"]
life_exp = life_exp.reset_index()

In [118]:
population = population.set_index("country", drop=True)
population[["1800"]] = population[["1800"]].fillna(0, axis=0)
population = population.ffill(axis=1)
population = (population.replace({'k':'*1000', "M":"*1000000","B":"*1000000000"}, regex=True).dropna().apply(pd.eval, axis=1, result_type="expand"))
a = (population.iloc[:,:100].replace({'k':'*1000', "M":"*1000000"}, regex=True).dropna().apply(pd.eval, axis=1, result_type="expand"))
a.columns = [i for i in range(1800,1900)]
b = (population.iloc[:,100:200].replace({'k':'*1000', "M":"*1000000","B":"*1000000000"}, regex=True).dropna().apply(pd.eval, axis=1, result_type="expand"))
b.columns = [i for i in range(1900,2000)]
c = (population.iloc[:,200:300].replace({'k':'*1000', "M":"*1000000","B":"*1000000000"}, regex=True).dropna().apply(pd.eval, axis=1, result_type="expand"))
c.columns = [i for i in range(2000,2100)]
d = (population.iloc[:,300::].replace({'k':'*1000', "M":"*1000000","B":"*1000000000"}, regex=True).dropna().apply(pd.eval, axis=1, result_type="expand"))
d.columns = [2100]
population = pd.concat([a,b,c,d],axis=1)
population = pd.DataFrame(population.stack())
population.index = population.index.rename('year', level=1)
population.columns = ["population"]
population = population.reset_index()

AttributeError: 'PandasExprVisitor' object has no attribute 'visit_Ellipsis'

In [216]:
# Merge dfs
df = population.merge(life_exp, how="left", on=["country", "year"])

df = df.merge(gni_capita, how="outer", on=["country", "year"])

df = df.fillna(0)

In [217]:
df

country  year  GNI per capita  life expectancy  population
31    Afghanistan  1990             0.0             53.8  12400000.0
32    Afghanistan  1991             0.0             53.8  13300000.0
33    Afghanistan  1992             0.0             54.2  14500000.0
34    Afghanistan  1993             0.0             54.4  15800000.0
35    Afghanistan  1994             0.0             53.9  17100000.0
...           ...   ...             ...              ...         ...
6102     Zimbabwe  2016          2560.0             59.2  14000000.0
6103     Zimbabwe  2017          3210.0             59.9  14200000.0
6104     Zimbabwe  2018          3370.0             60.6  14400000.0
6105     Zimbabwe  2019          3100.0             61.0  14600000.0
6106     Zimbabwe  2020          2850.0             60.7  14900000.0

[5828 rows x 5 columns]

In [218]:
df.to_csv("df_final.csv")

In [ ]:
# Merge dfs
df = gni_capita.merge(life_exp, how="left", on=["country", "year"])
df = df.merge(population, how="left", on=["country", "year"])
df = df.dropna()

In [ ]:
df.to_csv("df_final_filtered.csv")